In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
seinfeld = pd.read_csv('seinfeld episode rankings.csv')

In [3]:
seinfeld

,Title,Plot,Quotability/cultural impact,Laughs,Total score,IMDb rating,IMDb rank,Notes,Season,episode
0,The Marine Biologist,5.0,9.0,52.0,101.739130,9.2,4.0,NaN,5.0,14.0
1,The Contest 4:11,5.0,10.0,31.0,67.391304,9.6,1.0,first appearance of Estelle.,4.0,11.0
2,The Conversion,5.0,9.0,30.0,58.695652,8.1,52.0,NaN,5.0,11.0
3,The Junior Mint,4.0,10.0,27.0,46.956522,8.7,11.0,"""DOLORES!!!""",4.0,20.0
4,The Sniffing Accountant,3.5,10.0,27.5,41.847826,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0
...,...,...,...,...,...,...,...,...,...,...
174,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
175,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
176,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
177,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


In [4]:
seinfeld.columns=['title','plot','quotability','laughs','total_score','imdb_rating','imdb_rank','notes','season','episode']

In [5]:
seinfeld=seinfeld[seinfeld.laughs>0]

In [6]:
seinfeld.sort_values('total_score',ascending=False)

,title,plot,quotability,laughs,total_score,imdb_rating,imdb_rank,notes,season,episode
0,The Marine Biologist,5.0,9.0,52.0,101.739130,9.2,4.0,NaN,5.0,14.0
1,The Contest 4:11,5.0,10.0,31.0,67.391304,9.6,1.0,first appearance of Estelle.,4.0,11.0
2,The Conversion,5.0,9.0,30.0,58.695652,8.1,52.0,NaN,5.0,11.0
3,The Junior Mint,4.0,10.0,27.0,46.956522,8.7,11.0,"""DOLORES!!!""",4.0,20.0
4,The Sniffing Accountant,3.5,10.0,27.5,41.847826,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0
...,...,...,...,...,...,...,...,...,...,...
75,The Dog 3:4,3.0,3.0,7.0,2.739130,7.3,83.0,NaN,3.0,4.0
76,The Bris,1.0,5.0,12.0,2.608696,7.9,67.0,NaN,5.0,5.0
77,The Stock Tip 1:5,3.0,3.0,6.5,2.543478,7.5,78.0,NaN,1.0,5.0
78,The Robbery 1:4,3.0,3.5,5.5,2.510870,7.6,76.0,NaN,1.0,4.0


In [7]:
scripts = pd.read_csv('seinfeld_data/scripts.csv')

In [8]:
episodes = pd.read_csv('seinfeld_data/episode_info.csv')

In [9]:
episodes = episodes.drop(columns='Unnamed: 0')

In [10]:
episodes.Title = episodes.Title.astype(str)

In [11]:
episodes.Title = episodes.Title.str.lower().str.replace(' ','_').str.replace(',','')

In [12]:
scripts = scripts.drop(columns='Unnamed: 0')

In [13]:
scripts[0:211].SEID = 'S01E00'
scripts[0:211].EpisodeNo = 0.0

In [14]:
episodes

,Season,EpisodeNo,Title,AirDate,Writers,Director,SEID
0,1.0,1.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E01
1,1.0,1.0,the_stakeout,"May 31, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E01
2,1.0,2.0,the_robbery,"June 7, 1990",Matt Goldman,Tom Cherones,S01E02
3,1.0,3.0,male_unbonding,"June 14, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E03
4,1.0,4.0,the_stock_tip,"June 21, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E04
...,...,...,...,...,...,...,...
169,9.0,17.0,the_bookstore,"April 9, 1998",Spike Feresten,Andy Ackerman,S09E17
170,9.0,18.0,the_frogger,"April 23, 1998","Gregg Kavet, Andy Robin",Andy Ackerman,S09E18
171,9.0,19.0,the_maid,"April 30, 1998","Alec Berg, David Mandel, Jeff Schaffer",Andy Ackerman,S09E19
172,9.0,20.0,the_puerto_rican_day,"May 7, 1998","Alec Berg, Jennifer Crittenden, Spike Feresten...",Andy Ackerman,S09E20


In [15]:
episodes.SEID[0] = 'S01E00'
episodes.EpisodeNo[0] = 0.0

In [16]:
scripts

,Character,Dialogue,EpisodeNo,SEID,Season
0,JERRY,Do you know what this is all about? Do you kno...,0.0,S01E00,1.0
1,JERRY,"(pointing at Georges shirt) See, to me, that b...",0.0,S01E00,1.0
2,GEORGE,Are you through?,0.0,S01E00,1.0
3,JERRY,"You do of course try on, when you buy?",0.0,S01E00,1.0
4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",0.0,S01E00,1.0
...,...,...,...,...,...
54611,JERRY,Grand theft auto - don't steal any of my jokes.,23.0,S09E23,9.0
54612,PRISONER 3,You suck - I'm gonna cut you.,23.0,S09E23,9.0
54613,JERRY,"Hey, I don't come down to where you work, and ...",23.0,S09E23,9.0
54614,GUARD,"Alright, Seinfeld, that's it. Let's go. Come on.",23.0,S09E23,9.0


In [17]:
scripts[scripts.SEID=='S01E00']

,Character,Dialogue,EpisodeNo,SEID,Season
0,JERRY,Do you know what this is all about? Do you kno...,0.0,S01E00,1.0
1,JERRY,"(pointing at Georges shirt) See, to me, that b...",0.0,S01E00,1.0
2,GEORGE,Are you through?,0.0,S01E00,1.0
3,JERRY,"You do of course try on, when you buy?",0.0,S01E00,1.0
4,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",0.0,S01E00,1.0
...,...,...,...,...,...
206,LAURA,"Yeah, yeah, hes a great guy...",0.0,S01E00,1.0
207,JERRY,Yeah.,0.0,S01E00,1.0
208,LAURA,"Youd really like him, you know, I cant wait to...",0.0,S01E00,1.0
209,JERRY,Me too!,0.0,S01E00,1.0


In [18]:
scripts.Character=scripts.Character.str.replace(r"\(.*\)","")

In [19]:
line_count=scripts.Character.value_counts().head(475)

In [20]:
scripts.Character.str.strip().value_counts()

JERRY                                                                                               14905
GEORGE                                                                                               9781
ELAINE                                                                                               8073
KRAMER                                                                                               6723
NEWMAN                                                                                                649
                                                                                                    ...  
(ELAINE finally notices them, but suddenly from the other direction, three other guys are coming        1
ALL THREE                                                                                               1
JERR                                                                                                    1
Dx                                            

In [21]:
episodes["SEID"] = episodes.SEID.astype(str)
scripts["SEID"] = scripts.SEID.astype(str)

In [22]:
scripts[scripts.SEID=='S01E03']

,Character,Dialogue,EpisodeNo,SEID,Season
837,JERRY,"Most men like working on things. Tools, object...",3.0,S01E03,1.0
838,GEORGE,I had to say something. (chuckles) I had to sa...,3.0,S01E03,1.0
839,JERRY,I dont think you did anything wrong.,3.0,S01E03,1.0
840,GEORGE,I told her I liked her. Why? Why did I tell he...,3.0,S01E03,1.0
841,JERRY,We can only thank God for that.,3.0,S01E03,1.0
...,...,...,...,...,...
1073,JOEL,"You know, I should really try something like t...",3.0,S01E03,1.0
1074,JERRY,You really should.,3.0,S01E03,1.0
1075,JOEL,"Well, lets just take a look here.",3.0,S01E03,1.0
1076,JOEL,Forty-one home games. Let's see saturday night...,3.0,S01E03,1.0


In [23]:
pd.merge(episodes, right=scripts, on='SEID')

,Season_x,EpisodeNo_x,Title,AirDate,Writers,Director,SEID,Character,Dialogue,EpisodeNo_y,Season_y
0,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,JERRY,Do you know what this is all about? Do you kno...,0.0,1.0
1,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,JERRY,"(pointing at Georges shirt) See, to me, that b...",0.0,1.0
2,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,GEORGE,Are you through?,0.0,1.0
3,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,JERRY,"You do of course try on, when you buy?",0.0,1.0
4,1.0,0.0,good_news_bad_news,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00,GEORGE,"Yes, it was purple, I liked it, I dont actuall...",0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
54611,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,JERRY,Grand theft auto - don't steal any of my jokes.,23.0,9.0
54612,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,PRISONER 3,You suck - I'm gonna cut you.,23.0,9.0
54613,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,JERRY,"Hey, I don't come down to where you work, and ...",23.0,9.0
54614,9.0,23.0,the_finale,"May 14, 1998",Larry David,Andy Ackerman,S09E23,GUARD,"Alright, Seinfeld, that's it. Let's go. Come on.",23.0,9.0


In [24]:
seinfeld.title[0] = 'good_news_bad_news'

In [25]:
seinfeld

,title,plot,quotability,laughs,total_score,imdb_rating,imdb_rank,notes,season,episode
0,good_news_bad_news,5.0,9.0,52.0,101.739130,9.2,4.0,NaN,5.0,14.0
1,The Contest 4:11,5.0,10.0,31.0,67.391304,9.6,1.0,first appearance of Estelle.,4.0,11.0
2,The Conversion,5.0,9.0,30.0,58.695652,8.1,52.0,NaN,5.0,11.0
3,The Junior Mint,4.0,10.0,27.0,46.956522,8.7,11.0,"""DOLORES!!!""",4.0,20.0
4,The Sniffing Accountant,3.5,10.0,27.5,41.847826,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0
...,...,...,...,...,...,...,...,...,...,...
75,The Dog 3:4,3.0,3.0,7.0,2.739130,7.3,83.0,NaN,3.0,4.0
76,The Bris,1.0,5.0,12.0,2.608696,7.9,67.0,NaN,5.0,5.0
77,The Stock Tip 1:5,3.0,3.0,6.5,2.543478,7.5,78.0,NaN,1.0,5.0
78,The Robbery 1:4,3.0,3.5,5.5,2.510870,7.6,76.0,NaN,1.0,4.0


In [26]:
seinfeld

,title,plot,quotability,laughs,total_score,imdb_rating,imdb_rank,notes,season,episode
0,good_news_bad_news,5.0,9.0,52.0,101.739130,9.2,4.0,NaN,5.0,14.0
1,The Contest 4:11,5.0,10.0,31.0,67.391304,9.6,1.0,first appearance of Estelle.,4.0,11.0
2,The Conversion,5.0,9.0,30.0,58.695652,8.1,52.0,NaN,5.0,11.0
3,The Junior Mint,4.0,10.0,27.0,46.956522,8.7,11.0,"""DOLORES!!!""",4.0,20.0
4,The Sniffing Accountant,3.5,10.0,27.5,41.847826,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0
...,...,...,...,...,...,...,...,...,...,...
75,The Dog 3:4,3.0,3.0,7.0,2.739130,7.3,83.0,NaN,3.0,4.0
76,The Bris,1.0,5.0,12.0,2.608696,7.9,67.0,NaN,5.0,5.0
77,The Stock Tip 1:5,3.0,3.0,6.5,2.543478,7.5,78.0,NaN,1.0,5.0
78,The Robbery 1:4,3.0,3.5,5.5,2.510870,7.6,76.0,NaN,1.0,4.0


In [27]:
episodes.columns = episodes.columns.str.lower()


In [28]:
seinfeld.title = seinfeld.title.str.replace(r"\d.*\d","").str.strip().str.replace(' ','_').str.lower()

In [29]:
pd.merge(seinfeld, right = episodes, on='title')

,title,plot,quotability,laughs,total_score,imdb_rating,imdb_rank,notes,season_x,episode,season_y,episodeno,airdate,writers,director,seid
0,good_news_bad_news,5.0,9.0,52.0,101.739130,9.2,4.0,NaN,5.0,14.0,1.0,0.0,"July 5, 1989","Larry David, Jerry Seinfeld",Art Wolff,S01E00
1,the_contest,5.0,10.0,31.0,67.391304,9.6,1.0,first appearance of Estelle.,4.0,11.0,4.0,11.0,"November 18, 1992",Larry David,Tom Cherones,S04E11
2,the_conversion,5.0,9.0,30.0,58.695652,8.1,52.0,NaN,5.0,11.0,5.0,11.0,"December 16, 1993",Bruce Kirschbaum,Tom Cherones,S05E11
3,the_junior_mint,4.0,10.0,27.0,46.956522,8.7,11.0,"""DOLORES!!!""",4.0,20.0,4.0,20.0,"March 18, 1993",Andy Robin,Tom Cherones,S04E20
4,the_sniffing_accountant,3.5,10.0,27.5,41.847826,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0,5.0,4.0,"October 7, 1993","Larry David, Jerry Seinfeld",Tom Cherones,S05E04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,the_dog,3.0,3.0,7.0,2.739130,7.3,83.0,NaN,3.0,4.0,3.0,4.0,"October 9, 1991",Larry David,Tom Cherones,S03E04
64,the_bris,1.0,5.0,12.0,2.608696,7.9,67.0,NaN,5.0,5.0,5.0,5.0,"October 14, 1993",Larry Charles,Tom Cherones,S05E05
65,the_stock_tip,3.0,3.0,6.5,2.543478,7.5,78.0,NaN,1.0,5.0,1.0,4.0,"June 21, 1990","Larry David, Jerry Seinfeld",Tom Cherones,S01E04
66,the_robbery,3.0,3.5,5.5,2.510870,7.6,76.0,NaN,1.0,4.0,1.0,2.0,"June 7, 1990",Matt Goldman,Tom Cherones,S01E02


In [30]:
seinfeld.sort_values('total_score',ascending=False).head(30)

,title,plot,quotability,laughs,total_score,imdb_rating,imdb_rank,notes,season,episode
0,good_news_bad_news,5.0,9.0,52.0,101.739130,9.2,4.0,NaN,5.0,14.0
1,the_contest,5.0,10.0,31.0,67.391304,9.6,1.0,first appearance of Estelle.,4.0,11.0
2,the_conversion,5.0,9.0,30.0,58.695652,8.1,52.0,NaN,5.0,11.0
3,the_junior_mint,4.0,10.0,27.0,46.956522,8.7,11.0,"""DOLORES!!!""",4.0,20.0
4,the_sniffing_accountant,3.5,10.0,27.5,41.847826,8.0,60.0,"""heres to feeling good all the time""",5.0,4.0
5,the_stall,4.0,8.0,29.0,40.347826,8.4,24.0,NaN,5.0,12.0
6,the_pick,4.0,9.0,24.0,37.565217,8.5,21.0,NaN,4.0,13.0
7,the_pitch,5.0,10.0,16.5,35.869565,8.6,17.0,People like to say salsa,4.0,3.0
8,the_keys,5.0,7.5,20.5,33.423913,8.4,24.0,NaN,3.0,23.0
9,the_pie,4.0,8.0,23.0,32.000000,8.4,24.0,NaN,5.0,15.0
